In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("/home/exalis/GitHub/velib-prediction/velib-prediction/3. Modeling Research/datasets/monthly-mean-temp.csv")

In [3]:
df.head()

,Month,Temperature
0,1920-01,40.6
1,1920-02,40.8
2,1920-03,44.4
3,1920-04,46.7
4,1920-05,54.1


In [1]:
# grid search ets hyperparameters for monthly mean temp dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# one-step Holt Winters Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
    t,d,s,p,b,r = config
    # define model
    history = array(history)
    model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
    # fit model
    model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

In [5]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [6]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = exp_smoothing_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = measure_rmse(test, predictions)
    return error

In [7]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
    # one failure during model validation suggests an unstable config
        try:
        # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print( ' > Model[%s] %.3f ' % (key, result))
    return (key, result)

In [8]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
    scores = None
    if parallel:
    # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend= 'multiprocessing' )
        tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
    # remove empty results
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

In [9]:
if __name__ == ' __main__ ' :
    # load dataset
    series = read_csv("/home/exalis/GitHub/velib-prediction/velib-prediction/3. Modeling Research/datasets/monthly-mean-temp.csv", header=0, index_col=0)
    data = series.values
    # trim dataset to 5 years
    data = data[-(5*12):]
    # data split
    n_test = 12
    # model configs
    cfg_list = exp_smoothing_configs(seasonal=[0,12])
    # grid search
    scores = grid_search(data[:,0], cfg_list, n_test)
    print( ' done ' )
    # list top 3 configs
    for cfg, error in scores[:3]:
        print(cfg, error)

In [29]:
# OK
# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
    models = list()
    # define config lists
    t_params = ['add','mul', None]
    d_params = [True, False]
    s_params = ['add','mul', None]
    p_params = seasonal
    b_params = [True, False]
    r_params = [True, False]
    # create config instances
    for t in t_params:
        for d in d_params:
            for s in s_params:
                for p in p_params:
                    for b in b_params:
                        for r in r_params:
                            cfg = [t,d,s,p,b,r]
                            models.append(cfg)
    return models

# Creating list of configuration possible
cfg_list = exp_smoothing_configs(seasonal=[0,12])


In [30]:
cfg_list = exp_smoothing_configs(seasonal=[0,12])

In [11]:
series = read_csv("/home/exalis/GitHub/velib-prediction/velib-prediction/3. Modeling Research/datasets/monthly-mean-temp.csv", header=0, index_col=0)

In [12]:
data = series.values

In [13]:
data = data[-(5*12):]

In [14]:
n_test = 12

In [53]:
# one-step Holt Winters Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
    t,d,s,p,b,r = config
    # define model
    history = array(history)
    model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=12)
    # Modify the number of seasonel periods by p
    # fit model
    model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

In [15]:
exp_smoothing_forecast(history, cfg_list[0])

NameError: name 'history' is not defined

In [59]:
df.head()

,Month,Temperature
0,1920-01,40.6
1,1920-02,40.8
2,1920-03,44.4
3,1920-04,46.7
4,1920-05,54.1


In [18]:
# Remonthing the month

df.pop("Month")
df

,Temperature
0,40.6
1,40.8
2,44.4
3,46.7
4,54.1
...,...
235,61.8
236,58.2
237,46.7
238,46.6


In [19]:
X = np.array(df["Temperature"])

In [60]:
X_train, X_test = train_test_split(X, test_size=0.3, random_state=0)

In [61]:
X_test.shape

(72,)

In [62]:
X_train.shape

(168,)

In [83]:
model = ExponentialSmoothing(X_train, trend="add", damped=True, seasonal="add", seasonal_periods=12)

In [84]:
model_fit = model.fit(optimized=True, use_boxcox=True, remove_bias=False)

/home/exalis/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/holtwinters.py:712: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning)


In [88]:
yhat = model_fit.predict(len(X_train))

In [89]:
yhat

array([nan])

In [48]:
len(X_test)

168

In [44]:
len(X_test)

168